In [1]:
# Importing Libraries

In [2]:
import pandas as pd
import numpy as np

In [3]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [4]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [5]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [6]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [7]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [8]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [9]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

In [10]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [11]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [12]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [13]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [14]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [15]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

C:\Users\LENOVO\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


In [16]:
len(X_train[0][0])

9

In [17]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


- Defining the Architecture of LSTM

In [26]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [27]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [1]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

In [24]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 512        0        25        0                   0   
SITTING                  3      410        75        0                   0   
STANDING                 0       87       445        0                   0   
WALKING                  0        0         0      481                   2   
WALKING_DOWNSTAIRS       0        0         0        0                 382   
WALKING_UPSTAIRS         0        0         0        2                  18   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            3  
STANDING                           0  
WALKING                           13  
WALKING_DOWNSTAIRS                38  
WALKING_UPSTAIRS                 451  


In [27]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 4s 2ms/step


In [28]:
score

[0.3087582236972612, 0.9097387173396675]

- With a simple 2 layer architecture we got 90.09% accuracy and a loss of 0.30
- We can further imporve the performace with Hyperparameter tuning

# Assignment:

### Model-1:

In [17]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(40, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 40)                8000      
_________________________________________________________________
dropout_1 (Dropout)          (None, 40)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 246       
Total params: 8,246
Trainable params: 8,246
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [19]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 123s 17ms/step - loss: 1.2862 - acc: 0.4539 - val_loss: 1.1677 - val_acc: 0.5059
Epoch 2/30
7352/7352 [==============================] - 121s 16ms/step - loss: 1.1326 - acc: 0.5135 - val_loss: 1.0953 - val_acc: 0.5161
Epoch 3/30
7352/7352 [==============================] - 118s 16ms/step - loss: 0.9043 - acc: 0.6050 - val_loss: 0.9573 - val_acc: 0.6050
Epoch 4/30
7352/7352 [==============================] - 124s 17ms/step - loss: 0.7925 - acc: 0.6255 - val_loss: 1.1005 - val_acc: 0.5249
Epoch 5/30
7352/7352 [==============================] - 131s 18ms/step - loss: 0.7313 - acc: 0.6461 - val_loss: 0.8016 - val_acc: 0.6237
Epoch 6/30
7352/7352 [==============================] - 132s 18ms/step - loss: 0.7000 - acc: 0.6665 - val_loss: 0.9460 - val_acc: 0.5989
Epoch 7/30
7352/7352 [==============

In [20]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0         0        0                   0   
SITTING                  0      405        62        0                   5   
STANDING                 0      106       424        1                   0   
WALKING                  0        0         7      406                  27   
WALKING_DOWNSTAIRS       0        0         0        0                 417   
WALKING_UPSTAIRS         0        0         0        1                  11   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            27  
SITTING                           19  
STANDING                           1  
WALKING                           56  
WALKING_DOWNSTAIRS                 3  
WALKING_UPSTAIRS                 459  


In [21]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 8s 3ms/step


In [22]:
score

[0.5031543293462738, 0.8893790295215473]

### Model-2:

In [47]:
# code from https://keras.io/regularizers/
from keras.regularizers import L1L2
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from keras.layers import LSTM , BatchNormalization
reg = L1L2(0.01, 0.01)

In [42]:
model = Sequential()
model.add(LSTM(100, input_shape=(timesteps, input_dim), kernel_initializer='glorot_normal' , return_sequences=True, bias_regularizer=reg))
model.add(BatchNormalization())
model.add(Dropout(0.70))
model.add(LSTM(50))
model.add(Dropout(0.70))
model.add(Dense(n_classes, activation='sigmoid'))
print("Model Summary: ")
model.summary()

Model Summary: 
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 128, 100)          44000     
_________________________________________________________________
batch_normalization_4 (Batch (None, 128, 100)          400       
_________________________________________________________________
dropout_7 (Dropout)          (None, 128, 100)          0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dropout_8 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 306       
Total params: 74,906
Trainable params: 74,706
Non-trainable params: 200
________________________________

In [43]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
checkpoint_3 = ModelCheckpoint("model_6.h5",monitor="val_acc",mode="max",save_best_only = True,verbose=1)

In [44]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=20,callbacks=[checkpoint_3])

Train on 7352 samples, validate on 2947 samples
Epoch 1/20
7352/7352 [==============================] - 248s 34ms/step - loss: 2.3953 - acc: 0.5909 - val_loss: 1.8404 - val_acc: 0.5999

Epoch 00001: val_acc improved from -inf to 0.59993, saving model to model_6.h5
Epoch 2/20
7352/7352 [==============================] - 235s 32ms/step - loss: 1.1531 - acc: 0.6884 - val_loss: 1.0318 - val_acc: 0.5999

Epoch 00002: val_acc did not improve from 0.59993
Epoch 3/20
7352/7352 [==============================] - 232s 32ms/step - loss: 0.5914 - acc: 0.7877 - val_loss: 0.4268 - val_acc: 0.8660

Epoch 00003: val_acc improved from 0.59993 to 0.86597, saving model to model_6.h5
Epoch 4/20
7352/7352 [==============================] - 232s 32ms/step - loss: 0.4100 - acc: 0.8750 - val_loss: 0.3189 - val_acc: 0.8968

Epoch 00004: val_acc improved from 0.86597 to 0.89684, saving model to model_6.h5
Epoch 5/20
7352/7352 [==============================] - 234s 32ms/step - loss: 0.3255 - acc: 0.9000 - val_l

In [48]:
model = load_model('model_6.h5')

In [49]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  0      389        98        0                   0   
STANDING                 0       47       482        3                   0   
WALKING                  0        0         0      475                  20   
WALKING_DOWNSTAIRS       0        0         0        0                 420   
WALKING_UPSTAIRS         0        0         0        8                  14   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            4  
STANDING                           0  
WALKING                            1  
WALKING_DOWNSTAIRS                 0  
WALKING_UPSTAIRS                 449  


In [50]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 17s 6ms/step


In [51]:
score

[0.2544564464753497, 0.9338310145911096]

### Model-3:

In [32]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(70, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.7))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 70)                22400     
_________________________________________________________________
dropout_4 (Dropout)          (None, 70)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 426       
Total params: 22,826
Trainable params: 22,826
Non-trainable params: 0
_________________________________________________________________


In [33]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [34]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 126s 17ms/step - loss: 1.2871 - acc: 0.4425 - val_loss: 1.2365 - val_acc: 0.4669
Epoch 2/30
7352/7352 [==============================] - 137s 19ms/step - loss: 1.0508 - acc: 0.5423 - val_loss: 1.0076 - val_acc: 0.6088
Epoch 3/30
7352/7352 [==============================] - 136s 18ms/step - loss: 0.8036 - acc: 0.6270 - val_loss: 0.8368 - val_acc: 0.5979
Epoch 4/30
7352/7352 [==============================] - 134s 18ms/step - loss: 0.7237 - acc: 0.6608 - val_loss: 0.7729 - val_acc: 0.6159
Epoch 5/30
7352/7352 [==============================] - 136s 18ms/step - loss: 0.9305 - acc: 0.6065 - val_loss: 0.8180 - val_acc: 0.5826
Epoch 6/30
7352/7352 [==============================] - 135s 18ms/step - loss: 0.6588 - acc: 0.7073 - val_loss: 0.6039 - val_acc: 0.7238
Epoch 7/30
7352/7352 [==============================] - 136s 18ms/step - loss: 0.5586 - acc: 0.7560 - val_loss: 0.5590 - val_acc: 

In [35]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 485       13        27        1                   9   
SITTING                  0      373       111        4                   0   
STANDING                 0       81       450        0                   0   
WALKING                  0        0         0      462                  19   
WALKING_DOWNSTAIRS       0        0         0        1                 416   
WALKING_UPSTAIRS         0        0         0        2                   6   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             2  
SITTING                            3  
STANDING                           1  
WALKING                           15  
WALKING_DOWNSTAIRS                 3  
WALKING_UPSTAIRS                 463  


In [36]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 8s 3ms/step


In [37]:
score

[0.46246355840847214, 0.8988802171700034]

### Model-4:

In [23]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(50, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 50)                12000     
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 306       
Total params: 12,306
Trainable params: 12,306
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [25]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 125s 17ms/step - loss: 1.1955 - acc: 0.4834 - val_loss: 0.8875 - val_acc: 0.6159
Epoch 2/30
7352/7352 [==============================] - 125s 17ms/step - loss: 0.8413 - acc: 0.6178 - val_loss: 0.7971 - val_acc: 0.6057
Epoch 3/30
7352/7352 [==============================] - 135s 18ms/step - loss: 0.7409 - acc: 0.6605 - val_loss: 0.7410 - val_acc: 0.6810
Epoch 4/30
7352/7352 [==============================] - 134s 18ms/step - loss: 0.7369 - acc: 0.6862 - val_loss: 1.2686 - val_acc: 0.4058
Epoch 5/30
7352/7352 [==============================] - 134s 18ms/step - loss: 0.5986 - acc: 0.7520 - val_loss: 0.5492 - val_acc: 0.7652
Epoch 6/30
7352/7352 [==============================] - 134s 18ms/step - loss: 0.4807 - acc: 0.8075 - val_loss: 0.5612 - val_acc: 0.7727
Epoch 7/30
7352/7352 [==============================] - 135s 18ms/step - loss: 0.4437 - acc: 0.8478 - val_loss: 0.5127 - val_acc: 

In [26]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0        27        0                   0   
SITTING                  0      379       112        0                   0   
STANDING                 0       70       461        1                   0   
WALKING                  0        0         0      472                   0   
WALKING_DOWNSTAIRS       0        0         0        5                 378   
WALKING_UPSTAIRS         0        1         0        4                   0   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            0  
STANDING                           0  
WALKING                           24  
WALKING_DOWNSTAIRS                37  
WALKING_UPSTAIRS                 466  


In [27]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 8s 3ms/step


In [28]:
score

[0.4163782881195881, 0.9046487953851374]

### Model-5:

In [53]:
model = Sequential()
model.add(LSTM(100, input_shape=(timesteps, input_dim), kernel_initializer='glorot_normal' , return_sequences=True, bias_regularizer=reg))
model.add(BatchNormalization())
model.add(Dropout(0.70))
model.add(LSTM(50))
model.add(Dropout(0.70))
model.add(Dense(n_classes, activation='sigmoid'))
print("Model Summary: ")
model.summary()

Model Summary: 
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 128, 100)          44000     
_________________________________________________________________
batch_normalization_5 (Batch (None, 128, 100)          400       
_________________________________________________________________
dropout_9 (Dropout)          (None, 128, 100)          0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 50)                30200     
_________________________________________________________________
dropout_10 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 306       
Total params: 74,906
Trainable params: 74,706
Non-trainable params: 200
________________________________

In [56]:
# Compiling the model
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
checkpoint_3 = ModelCheckpoint("model_7.h5",monitor="val_acc",mode="max",save_best_only = True,verbose=1)

In [57]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=20,callbacks=[checkpoint_3])

Train on 7352 samples, validate on 2947 samples
Epoch 1/20
7352/7352 [==============================] - 244s 33ms/step - loss: 1.7097 - acc: 0.8483 - val_loss: 1.0533 - val_acc: 0.8947

Epoch 00001: val_acc improved from -inf to 0.89470, saving model to model_7.h5
Epoch 2/20
7352/7352 [==============================] - 249s 34ms/step - loss: 0.6515 - acc: 0.9053 - val_loss: 0.2413 - val_acc: 0.9536

Epoch 00002: val_acc improved from 0.89470 to 0.95357, saving model to model_7.h5
Epoch 3/20
7352/7352 [==============================] - 240s 33ms/step - loss: 0.1730 - acc: 0.9407 - val_loss: 0.1105 - val_acc: 0.9600

Epoch 00003: val_acc improved from 0.95357 to 0.96002, saving model to model_7.h5
Epoch 4/20
7352/7352 [==============================] - 243s 33ms/step - loss: 0.1270 - acc: 0.9586 - val_loss: 0.1410 - val_acc: 0.9484

Epoch 00004: val_acc did not improve from 0.96002
Epoch 5/20
7352/7352 [==============================] - 253s 34ms/step - loss: 0.1128 - acc: 0.9617 - val_l

In [58]:
model = load_model('model_7.h5')

In [59]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                 20      419        51        0                   0   
STANDING                 0      116       416        0                   0   
WALKING                  0        0         2      490                   3   
WALKING_DOWNSTAIRS       0        0         0        0                 420   
WALKING_UPSTAIRS         0        1         0       11                  10   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            1  
STANDING                           0  
WALKING                            1  
WALKING_DOWNSTAIRS                 0  
WALKING_UPSTAIRS                 449  


In [60]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 18s 6ms/step


In [61]:
score

[0.07431104327187013, 0.9751159407787497]

## Results(PrettyTable):

In [3]:
from prettytable import PrettyTable
x = PrettyTable()
x.field_names = ["Model","Description", "Test loss", "Test Accuracy"]
x.add_row(["1","1 Layer of LSTM(40)","0.5031", "0.8893"])
x.add_row(["2","2 layers of LSTM, BN, Categoricalcross_entropy","0.2544", "0.9338"])
x.add_row(["3","1 Layer of LSTM(70)","0.4624", "0.8988"])
x.add_row(["4","1 Layer of LSTM(50)","0.4163", "0.9046"])
x.add_row(["5","2 layers of LSTM, BN, Binarycross_entropy","0.0743", "0.9751"])
print(x)

+-------+------------------------------------------------+-----------+---------------+
| Model |                  Description                   | Test loss | Test Accuracy |
+-------+------------------------------------------------+-----------+---------------+
|   1   |              1 Layer of LSTM(40)               |   0.5031  |     0.8893    |
|   2   | 2 layers of LSTM, BN, Categoricalcross_entropy |   0.2544  |     0.9338    |
|   3   |              1 Layer of LSTM(70)               |   0.4624  |     0.8988    |
|   4   |              1 Layer of LSTM(50)               |   0.4163  |     0.9046    |
|   5   |   2 layers of LSTM, BN, Binarycross_entropy    |   0.0743  |     0.9751    |
+-------+------------------------------------------------+-----------+---------------+


## Conclusions:
1. First i've used a single LSTM layer with 40 units, and got 0.88
2. Then  i used 2 layers of LSTM  follwed by Batch normalizationn with categorical cross entropy and got 0.93
3. For the 3rd and 4th model ,i used 1 Layer of LSTM with 70,50 units respectively and got a accuracy of 0.90
4. Now For 5th model i've used 2 Layers of LSTM with Binary cross entropy along with Batch normalization,with some dropout and got accuracy of 0.97